# PTEN loss Survival Analysis for HNSCC

Imports

In [1]:
import pandas as pd
import cptac
import cptac.utils as ut
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import lifelines
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
from lifelines.statistics import proportional_hazard_test
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
'''
Params
cancer_object: Object. The loaded cancer data set.
all_prot: List. trans proteins used to get proteomics data. 
gene_in: String. Gene used to get mutation status for the Mutation col.
utils: utils package from cptac.  

Returns a dataframe with trans proteomics and mutation status of a specific gene.
'''

def all_prot_format_df(cancer_object, all_prot, gene_in = 'PTEN', utils = ut):
    mut_type = cancer_object.get_genotype_all_vars(gene_in)
    
    if cancer_object.get_cancer_type() not in ('luad', 'endometrial'):
        # Keep only tumor samples from proteomics
        prot_and_mutations = cancer_object.join_omics_to_mutations(
            mutations_genes = [gene_in], omics_df_name = 'proteomics', omics_genes = all_prot, tissue_type = 'tumor')
        # Reduce a multiindex 
        if isinstance(prot_and_mutations.keys(), pd.core.indexes.multi.MultiIndex):
            prot_and_mutations = utils.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)
            prot_and_mutations = rename_duplicate_cols(prot_and_mutations)
        
        # Merge Mutation column from get_genotype_all_vars (includes cnv) with proteomics
        mut_type = mut_type[['Mutation']] 
        prot_df = prot_and_mutations.iloc[:,:-4] # Keep only proteomics
        merged = prot_df.join(mut_type, how = 'outer')
        
        # Keep only Wildtype and deletion
        compare = ['Wildtype_Tumor','Deletion']
        get = merged['Mutation'].isin(compare)
        mut_wt = merged[get]
    
    
    # Luad has no somatic mutations for PTEN which changes some things
    elif cancer_object.get_cancer_type() in ('luad'): 
        # get_genotype_all_vars add cnv data under the column PTEN
        mut_type = mut_type.drop(columns= gene_in)
        # different code because no somatic mutation data for pten (can't join to somatic mutations)
        omics = cancer_object.join_omics_to_omics(df1_name = 'CNV', df2_name='proteomics',genes1= gene_in, 
            genes2= all_prot, tissue_type = 'tumor')
        omics = utils.reduce_multiindex(omics, levels_to_drop = 1, flatten = True)
        omics = rename_duplicate_cols(omics)
        omics = omics.drop(columns='PTEN_CNV')
       
        # Merge Mutation column from get_genotype_all_vars (includes cnv) with proteomics
        merged = omics.join(mut_type, how = 'outer') 
        # Keep only No_Mutation (wildtype) and deletion
        compare = ['No_Mutation','Deletion']
        get = merged['Mutation'].isin(compare)
        mut_wt = merged[get]
        mut_wt['Mutation'] = np.where(
            mut_wt['Mutation'] == 'No_Mutation', 'Wildtype_Tumor', 'Deletion')
        
    elif cancer_object.get_cancer_type() in ('endometrial'):
        # merge cnv with genotype all mut type
        prot_and_mutations = cancer_object.join_omics_to_mutations(
            mutations_genes = [gene_in], omics_df_name = 'proteomics', omics_genes = all_prot, tissue_type = 'tumor') # drop Normal samples

        prot_df = prot_and_mutations.iloc[:,:-4] #drop Mutation and Location cols
        mut_type = mut_type[['Mutation']] # Get Mutation col that includes CNV
        merged = prot_df.join(mut_type, how = 'outer') # merge 

        # Create Truncation category and keep truncation and wt
        compare = ['Wildtype_Tumor','Nonsense_Mutation','Frame_Shift_Ins','Frame_Shift_Del']
        get = merged['Mutation'].isin(compare)
        mut_wt = merged[get]
        mut_wt['Mutation'] = np.where(
                        mut_wt['Mutation'] == 'Wildtype_Tumor', 'Wildtype_Tumor', 'Truncation')
    
    mut_wt = mut_wt.rename(columns={col: re.sub(r'_proteomics', '', col) for col in mut_wt.columns.tolist()})
    return mut_wt


In [4]:
''' Adds '_i' to duplicate col names where the int i increases by one with each duplicate 
Returns a df with unique column names. '''

def rename_duplicate_cols(df):
    cols = pd.Series(df.columns[:])

    for dup in cols[cols.duplicated()].unique(): 
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]

    # rename the columns with the cols list.
    df.columns=cols
    return df

In [117]:
def get_survival_cols(cancer_object):
    cancer = cancer_object.get_cancer_type()
    #print(cancer)
    
    # get unique survival col names
    if cancer == 'gbm':
        cols = ['vital_status', 'path_diag_to_last_contact_days', 'path_diag_to_death_days']
    if cancer == 'hnscc':
        cols = ['VITAL_STATUS', 'PATHO_DIAG_TO_LAST_CONT_DAYS', 'PATHO_DIAG_TO_DEATH_DAYS']
    if cancer == 'luad':
        cols = ['Vital Status', 'Path Diag to Last Contact(Day)', 'Path Diag to Death(days)']
    if cancer == 'ovarian':
        cols = ['Vital_Status', 'Days_Between_Collection_And_Last_Contact', 'Days_Between_Collection_And_Death']
    if cancer == 'endometrial':
        cols = ['Vital Status', 'Path Diag to Last Contact(Day)', 'Path Diag to Death(days)']
    
    # lscc no follow up or clin matching survival cols
    # Need more cols
    if cancer == 'brca':
        cols = ['Vital Status (at time of last contact)'] 
        print('2 deaths - not enough data')
        return 0
    if cancer == 'colon':
        cols = ['Vital Status (at time of last contact)'] 
        # convert to days? Date of Last Contact (Do not answer if patient is deceased)', 'Date of Death'
        return 0
    
    return cols
    
    

In [234]:
def get_survival_df(cancer_object):
    cols = get_survival_cols(cancer_object)

    # Get cols from followup or clinical 
    data_in_cinical = ['gbm', 'ovarian']
    if cancer not in data_in_cinical:
        df = cancer_object.get_followup()
    else:
        df = cancer_object.get_clinical()
    
    # Make col names consistent and keep survival cols
    consistent = df.rename(columns = {cols[0]: 'vital_status', cols[1]:'days_to_last_contact',
                                        cols[2]: 'days_to_death'})
    renamed_df = consistent[['vital_status', 'days_to_last_contact', 'days_to_death']]
    
    
    '''
    concise_last_contact = get_most_recent_last_contact_df(renamed_df)
    print('num index concise_last_contact:', len(concise_last_contact.index))
    
    unique_index_df = renamed_df.drop_duplicates(subset = 'days_to_last_contact', keep = 'last')
    print('drop', len(unique_index_df))
    new_df = unique_index_df.drop(columns = ['days_to_last_contact'])
    survival_df = new_df.join(concise_last_contact, how = 'outer') # duplicate index are now duplicate in last_contact col
    print('num index survival_df:', len(survival_df.index))
    print(survival_df['vital_status'].value_counts())
    
    renamed_df = renamed_df.drop_duplicates()
    print(len(renamed_df))'''
        
    return renamed_df

In [123]:
def get_most_recent_last_contact_df(survival_in):
    d = {}
    df = survival_in[['days_to_last_contact']]
    index_list = list(df.index.unique())
    for i in index_list:
        one_index_df = df.loc[[i]]
        # Get the latest number when duplicate records
        max_list = list(one_index_df['days_to_last_contact'].max()) # get most recent (highest num)
        d[i] = max_list[0]
        
    last_contact_df = pd.DataFrame.from_dict(d, orient='index', columns=['days_to_last_contact'])
    
    return last_contact_df    

# Step 1: Import data

In [124]:
cancer = 'HNSCC'
ca = cptac.Hnscc()

In [258]:
# Know number of vital status
clin = ca.get_clinical(tissue_type = 'tumor')
clin.follow_up_vital_status.value_counts()

Living      87
Deceased    14
Name: follow_up_vital_status, dtype: int64

# Step 2: Get df with PTEN mutation status and clinical data

In [277]:
cols = ['follow_up_vital_status', 'follow_up_days']
df = clin[cols]
df = df.rename(columns = {cols[0]: 'vital_status', cols[1]:'days_to_last_contact_or_death'})
len(df)
df

Name,vital_status,days_to_last_contact_or_death
Patient_ID,,
C3L-00977,Living,377.0
C3L-00987,Living,429.0
C3L-00994,Deceased,132.0
C3L-00995,Deceased,-10.0
C3L-00997,Living,442.0
...,...,...
C3N-04277,Living,436.0
C3N-04278,Living,351.0
C3N-04279,Living,310.0


In [278]:
#df = get_survival_df(ca) # get 'vital_status', 'days_to_last_contact', 'days_to_death'

del_wt = all_prot_format_df(ca, ['PTEN']) # get Mutation col (PTEN del/wt)
len(del_wt)
mut_survival = del_wt.join(df, on = 'Patient_ID', how = 'left') # combine keeping only turmors
len(mut_survival)
mut_survival

Name,PTEN,Mutation,vital_status,days_to_last_contact_or_death
Patient_ID,,,,
C3L-00977,23.604974,Wildtype_Tumor,Living,377.0
C3L-00987,23.370790,Deletion,Living,429.0
C3L-00994,23.924803,Wildtype_Tumor,Deceased,132.0
C3L-00995,23.961489,Wildtype_Tumor,Deceased,-10.0
C3L-00997,23.319454,Wildtype_Tumor,Living,442.0
...,...,...,...,...
C3N-04277,24.097503,Wildtype_Tumor,Living,436.0
C3N-04278,23.635862,Wildtype_Tumor,Living,351.0
C3N-04279,23.918302,Wildtype_Tumor,Living,310.0


In [270]:
stat = ca.get_clinical()
stat.columns
stat.Sample_Tumor_Normal.value_counts()

Tumor     111
Normal     81
Name: Sample_Tumor_Normal, dtype: int64

# Step 3: Kaplan Meier Plotting

Kaplan Meier plots show the probability of some event occuring over a given length of time, based on some attribute(s). 

In [284]:
#Make the vital_status column boolean
mut_survival['vital_status'] = mut_survival['vital_status'].replace('Living', False)
mut_survival['vital_status'] = mut_survival['vital_status'].replace('Deceased', True)
mut_survival['vital_status'] = mut_survival['vital_status'].astype('bool')
df_genes = mut_survival
# Create one column with days between last contact or death
#days = ['days_to_last_contact', 'days_to_death']
#df_genes = mut_survival.assign(days_to_last_contact_or_death = mut_survival[days].sum(1)).drop(days, 1)

In [286]:
# Plot
time = df_genes['days_to_last_contact_or_death']
status = df_genes['vital_status']

kmf = KaplanMeierFitter()
kmf.fit(time, event_observed = status)
kmf.plot()

TypeError: NaNs were detected in the dataset. Try using pd.isnull to find the problematic values.

# Step 4: Prepare Data for Multivariate Kaplan Meier Plots and Cox's Proportional Hazard Test

In [ ]:
# map numeric values to correspond with mutation status
mutation_map = {"Wildtype_Tumor" : 0, "Deletion" : 1}

df_genes['PTEN'] = df_genes['Mutation'].map(mutation_map)

#Then we will drop missing values, as missing values 
#will throw an error in the functions we will use in lifelines.
df_clean = df_genes.dropna(axis=0, how='any').copy()
df_clean = df_clean.drop(columns= 'Mutation') # drop col with str values (info mapped to PTEN as int)
df_clean

In [ ]:
df_clean.loc[df_clean['days_to_last_contact_or_death'] < 500]

In [ ]:
gene_in = 'PTEN'
mut_type = ca.get_genotype_all_vars(gene_in)
len(mut_type)

In [ ]:
mut_type.Mutation.value_counts()

In [ ]:
all_prot = ['PTEN']

if ca.get_cancer_type() not in ('luad', 'endometrial'):
    # Keep only tumor samples from proteomics
    prot_and_mutations = ca.join_omics_to_mutations(
        mutations_genes = [gene_in], omics_df_name = 'proteomics', omics_genes = all_prot, tissue_type = 'tumor')
    # Reduce a multiindex 
    if isinstance(prot_and_mutations.keys(), pd.core.indexes.multi.MultiIndex):
        prot_and_mutations = u.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)
        #prot_and_mutations = rename_duplicate_cols(prot_and_mutations)

    # Merge Mutation column from get_genotype_all_vars (includes cnv) with proteomics
    mut_type = mut_type[['Mutation']] 
    prot_df = prot_and_mutations.iloc[:,:-4] # Keep only proteomics
    merged = prot_df.join(mut_type, how = 'outer')

In [ ]:
merged.index.value_counts()
merged.Mutation.value_counts()

In [ ]:
d = df_clean.loc[df_clean['PTEN'] == 2]
len(d)
d

Verify that your columns are the correct data types.  The event of interest, in this case Vital_Status needs to contain boolean values, and all other columns in the table must be of a numeric type (either int64 or float64).

In [ ]:
for col in df_clean.columns:
    print(col, ":", df_clean[col].dtype)

# Step 5: Plot Multivariate Survival Risk

1 is wt. 2 is mutant.

In [ ]:
cph = CoxPHFitter()
cph.fit(df_clean, duration_col = "days_to_last_contact_or_death", 
        event_col = "vital_status")

plot_title = cancer+" Survival Risk: PTEN loss"
cph.plot_partial_effects_on_outcome('PTEN', [1,2], cmap='coolwarm', title=plot_title)
    

In [ ]:
cph.print_summary(model="untransformed variables", decimals=3)


# Step 6: Cox's Proportional Hazard Test

With the proportional_hazard_test function, we can now perform Cox's Proportional Hazard Test on the data to determine how each attribute contributes to our cohort's overall survival. This is shown by the hazard ratio in the column labeled -log2(p) below. In general, a hazard ratio of 1 suggests that an attribute has no effect on overall survival. A ratio less than 1 suggests that an attribute contributes to lower survival risk. A ratio greater than 1 suggests that an attribute contributes to higher survival risk.

In [ ]:
results = proportional_hazard_test(cph, df_clean, time_transform='rank')
results.print_summary(decimals=3, model="untransformed variables")

Below, we show the confidence interval for the hazard ratio. 

In [ ]:
plt.title(cancer)
cph.plot()

In [ ]:
# log(hazards ratio) = coef = -0.192

Conclusions: Because the p-value was greater than 0.05, there is insufficient evidence to suggest the PTEN loss effects survival risk.